<a href="https://colab.research.google.com/github/Kling0012/youtubeSCRs/blob/main/YouTubelistSCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

获取频道所以播放列表&URL

In [ ]:
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import openpyxl

# Google APIクライアントを認証します。
api_key = ""
youtube = build("youtube", "v3", developerKey=api_key)

# チャンネルURLからチャンネルIDを取得します。
channel_url = "https://www.youtube.com/user/BYCMsince2012"
channel_id_request = youtube.channels().list(
    part="id",
    forUsername=channel_url.split("/")[-1],
)
channel_id_response = channel_id_request.execute()
channel_id = channel_id_response["items"][0]["id"]

# チャンネルIDから再生リストの一覧を取得します。
playlist_request = youtube.playlists().list(
    part="snippet,contentDetails",
    channelId=channel_id,
    maxResults=50,
)
playlist_response = playlist_request.execute()
playlist_infos = playlist_response["items"]
for i, playlist_info in enumerate(playlist_infos):
    title = playlist_info["snippet"]["title"]
    count = playlist_info["contentDetails"]["itemCount"]
    url = f"https://www.youtube.com/playlist?list={playlist_info['id']}"
    print(f"{i+1}. {title} ({count} videos) URL: {url}")


使用URL获取列表内视频信息

In [ ]:
import openpyxl
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


def get_videos(playlist_id, api_key):

    youtube = build('youtube', 'v3', developerKey=api_key)
    videos = []
    next_page_token = None

    while True:
        res = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos += res['items']
        next_page_token = res.get('nextPageToken')

        if not next_page_token:
            break

    return videos


def main():
    #API
    api_key = ''

    #列表URL
    playlist_url = 'https://www.youtube.com/playlist?list=PLl32tGbAYNEo78QFPFFoqNSnja3jNkyUu'
    playlist_id = playlist_url.split('=')[1]

    
    videos = get_videos(playlist_id, api_key)

    
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = 'videos'

    
    sheet['A1'] = 'title'
    sheet['B1'] = 'description'
    sheet['C1'] = 'thumbnail_url'
    sheet['D1'] = 'video_url'
    sheet['E1'] = 'published_at'
    sheet['F1'] = 'view_count'
    sheet['G1'] = 'duration'

    
    for i, video in enumerate(videos):
        video_id = video["snippet"]["resourceId"]["videoId"]
        video_url = f'https://www.youtube.com/watch?v={video_id}'

        video_details_request = youtube.videos().list(
            part="snippet,statistics,contentDetails",
            id=video_id,
        )
        video_details_response = video_details_request.execute()
        video_details = video_details_response["items"][0]["snippet"]

        title = video_details["title"]
        description = video_details["description"]
        thumbnail_url = video_details["thumbnails"]["default"]["url"]
        published_at = video_details["publishedAt"]
        view_count = video_details_response["items"][0]["statistics"]["viewCount"]
        duration = video_details_response["items"][0]["contentDetails"]["duration"]

        sheet.cell(row=i+2, column=1, value=title)
        sheet.cell(row=i+2, column=2, value=description)
        sheet.cell(row=i+2, column=3, value=thumbnail_url)
        sheet.cell(row=i+2, column=4, value=video_url)
        sheet.cell(row=i+2, column=5, value=published_at)
        sheet.cell(row=i+2, column=6, value=view_count)
        sheet.cell(row=i+2, column=7, value=duration)

    
    workbook.save('videos.xlsx')

if __name__ == '__main__':
    main()
